In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from tqdm import tqdm
from dateutil import parser
import string
import re
import time

In [62]:
# Scrape the latest Nifty50 company names, prices, change etc.
# At the moment, this is only scraping info required for news scraping, i'll scrape prices etc. at a later stage.

getnifty50 = "https://www.moneycontrol.com/stocks/marketstats/indcontrib.php?optex=NSE&opttopic=indcontrib&index=9"
soup = BeautifulSoup(get(getnifty50).text, 'lxml')
Company_Name = [script.text.strip() for script in soup.select("a.bl_12")[0::2]]
Industry = [script.text.strip() for script in soup.select("a.bl_12")[1::2]]

urlsplit1 = [script.get('href').split('/')[-2] for script in soup.select("a.bl_12")[0::2]]
urlsplit2 = [script.get('href').split('/')[-1] for script in soup.select("a.bl_12")[0::2]]

nifty50_lookuptable = pd.DataFrame({
    'Company_Name' : Company_Name,
    'Industry' : Industry,
    'urlsplit1' : urlsplit1,
    'urlsplit2' : urlsplit2
})

print(nifty50_lookuptable.shape)
nifty50_lookuptable.head()

(50, 4)


,Company_Name,Industry,urlsplit1,urlsplit2
0,HDFC Bank,Bank - Private,hdfcbank,HDF01
1,HDFC,Finance - Housing,hdfc,HDF
2,ICICI Bank,Bank - Private,icicibank,ICI02
3,Infosys,IT Services & Consulting,infosys,IT
4,Kotak Mahindra,Bank - Private,kotakmahindra,KMB


In [134]:
def initialize(urlsplit2):
    '''
    Function to obtain total number of result pages, initialize blank news data and
    set urls for moneycontrol news search page for 2020 and 2019.
    '''
    global ticker, url_all, headlines, dates, news, urls, sources
    
    url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id={}&scat=&pageno={}&next=0&durationType=Y&Year={}&duration=1&news_type="
    url19 = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=" + urlsplit2 + "&durationType=Y&Year=2019"
    url20 = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=" + urlsplit2 + "&durationType=Y&Year=2020"
    
    soup = BeautifulSoup(get(url19).text, 'lxml')
    result_max = [len(soup.select('div.pages.MR10.MT15')[0].select('a')) + 1]
    
    soup = BeautifulSoup(get(url20).text, 'lxml')
    ticker = soup.select('div.FL.gry10')[0].text.split('|')[1].split(':')[1].strip()
    result_max += [len(soup.select('div.pages.MR10.MT15')[0].select('a')) + 1]
    
    url_all = [url.format(ticker, i, 2019) for i in range(1, result_max[0]+1)] + [url.format(ticker, i, 2020) for i in range(1, result_max[0]+1)]
    headlines, dates, news, urls, sources = [], [], [], [], []
    print("Total number of result pages for", ticker, ":", result_max)

In [ ]:
# Yet to update the below codes for MoneyControl. Above part of the notebook code is already done.

In [4]:
def getnewslinks():
    '''
    Function to scrape news headlines, urls, publish dates etc.
    '''
    print("[INFO] Extracting Links...")

    for src in tqdm(url_all):

        try:
            soup = BeautifulSoup(get(src).text, 'lxml')

            # Extracts the Headlines
            try:
                headline = [script.text.strip() for script in soup.select('a.story-card75x1-text')]
                headlines.extend(headline)
            except:
                headlines.extend(None)

            # Extracts the urls
            try:
                source = [script.get('href') for script in soup.select('a.story-card75x1-text')]
                urls.extend(source)
            except:
                urls.extend(None)

            # Extracts the sections(markets, industry, business etc.)
            try:
                section = [script.text.strip() for script in soup.select('a.section-name')]
                sections.extend(section)
            except:
                sections.extend(None)

            # Extracts the published dates
            try:
                dateline = [str(parser.parse(script.text)).split()[0] for script in soup.select('span.dateline')]
                dates.extend(dateline)
            except:
                dates.extend(None)

            # Extracts the bylines
    #         try:
    #             bylines = [script.text.strip() for script in soup.select('a.story-card-33-author-name')]
    #             authors.extend(bylines)
    #             assert len(bylines)==12
    #         except:
    #             authors.extend(None)

        except:
            print("Exception occurred in url : ", src)
            break

    print("[INFO] Links Extracted.")
    print("Total No. of Pages to be Scraped = ", len(urls))
    print("Oldest Available Article: ", min(dates))

In [5]:
def getarticles():
    '''
    Function to scrape news articles and bylines.
    '''
    print("[INFO] Extracting Articles...")

    for src in tqdm(urls):
        try:
            # Parse the url to NewsPage
            soup = BeautifulSoup(get(src).text, 'lxml')

            # Extracts the news articles
            try:
                news_article = soup.find(id='content-body-14269002-' + re.findall(r"\d+",src.split('/')[-1])[0]).text.strip()
                news.append(news_article)
            except:
                news.append(None)

            # Extracts the bylines
            try:
                bylines = [script.text.strip() for script in soup.select('a.auth-nm')]
                authors.extend([' | '.join(bylines)])
            except:
                authors.extend(None)

        except:
            print("Exception occurred in url : ", src)
            news.append(None)

    print("[INFO] Articles Extracted.")

In [6]:
def chkdata():
    '''
    Function to check for any missing values in the Dataframe and drop it.
    '''
    global df
    df = pd.DataFrame({'Headlines': headlines,
                       'Sections' : sections,
                       'Articles': news,
                       'Published_Dates': dates,
                       'Source_URLs': urls,
                       'ByLines' : authors
                       })
    print("Missing Info in Scraped Data :")
    print(df.isna().sum())
    df=df.dropna(axis = 0)
    print("Total Usable Scraped Data : ", df.shape)

In [7]:
def savefile(SearchString):
    '''
    Function to save the scraped data as pickle file.
    '''
    sname = '_'.join(SearchString.split('%20'))
    # df.to_csv("news_thehindu_"+ sname + ".csv")
    df.to_pickle("news_thehindu_"+ sname + ".pkl")
    print("Data saved for", SearchString, ".")

In [ ]:
# Scraping news articles for all the companies listed in Nifty50

for i, sstring in enumerate(SearchString):
    print("Nifty50 Extraction Search Count :",i+1)
    initialize(sstring)
    getnewslinks()
    getarticles()
    chkdata()
    savefile(sstring)
    time.sleep(5)

Nifty50 Extraction Search Count : 1


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Total number of result pages for reliance%20petroleum : 6
[INFO] Extracting Links...


  2%|█▎                                                                                 | 1/63 [00:00<00:12,  4.78it/s]

[INFO] Links Extracted.
Total No. of Pages to be Scraped =  63
Oldest Available Article:  2019-10-23
[INFO] Extracting Articles...


...

  0%|                                                                                           | 0/35 [00:00<?, ?it/s]

Total number of result pages for axis%20bank : 35
[INFO] Extracting Links...


  0%|                                                                                          | 0/410 [00:00<?, ?it/s]

[INFO] Links Extracted.
Total No. of Pages to be Scraped =  410
Oldest Available Article:  2019-10-22
[INFO] Extracting Articles...


 86%|████████████████████████████████████████████████████████████████████▉           | 353/410 [01:42<00:15,  3.71it/s]

In [9]:
# Scraping news articles for all the companies listed in Nifty50

for i, sstring in enumerate(SearchString):
    if i >= 10:
        print("Nifty50 Extraction Search Count :",i+1)
        initialize(sstring)
        getnewslinks()
        getarticles()
        chkdata()
        savefile(sstring)
        time.sleep(5)

Nifty50 Extraction Search Count : 11


  0%|                                                                                           | 0/35 [00:00<?, ?it/s]

Total number of result pages for axis%20bank : 35
[INFO] Extracting Links...


  0%|                                                                                          | 0/410 [00:00<?, ?it/s]

[INFO] Links Extracted.
Total No. of Pages to be Scraped =  410
Oldest Available Article:  2019-10-22
[INFO] Extracting Articles...


...

  0%|                                                                                           | 0/68 [00:00<?, ?it/s]

[INFO] Links Extracted.
Total No. of Pages to be Scraped =  68
Oldest Available Article:  2019-10-24
[INFO] Extracting Articles...


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [00:37<00:00,  1.81it/s]


[INFO] Articles Extracted.
Missing Info in Scraped Data :
Headlines           0
Sections            0
Articles           13
Published_Dates     0
Source_URLs         0
ByLines             0
dtype: int64
Total Usable Scraped Data :  (55, 6)
Data saved for wipro .
Nifty50 Extraction Search Count : 26


ValueError: max() arg is an empty sequence

In [20]:
# Scraping news articles for all the companies listed in Nifty50

for i, sstring in enumerate(SearchString):
    if i >= 25:
        print("Nifty50 Extraction Search Count :",i+1)
        initialize(sstring)
        getnewslinks()
        getarticles()
        chkdata()
        savefile(sstring)
        time.sleep(5)

Nifty50 Extraction Search Count : 26


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Total number of result pages for britannia : 1
[INFO] Extracting Links...


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

[INFO] Links Extracted.
Total No. of Pages to be Scraped =  7
Oldest Available Article:  2019-12-17
[INFO] Extracting Articles...


...

Total number of result pages for gail : 3
[INFO] Extracting Links...


  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

[INFO] Links Extracted.
Total No. of Pages to be Scraped =  27
Oldest Available Article:  2020-01-08
[INFO] Extracting Articles...


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.22it/s]


[INFO] Articles Extracted.
Missing Info in Scraped Data :
Headlines          0
Sections           0
Articles           5
Published_Dates    0
Source_URLs        0
ByLines            0
dtype: int64
Total Usable Scraped Data :  (22, 6)
Data saved for gail .


In [ ]:
# The pages with blank Articles have a different structure.
# These pages have a byline 'The Hindu Net Desk' and can be separately scraped and preprocessed.